In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime
import time

actor_errors=0
show_errors=0
genre_errors=0
country_errors=0
relations_errors=0

In [2]:
actor_start_time = time.time()
show_start_time = time.time()
genre_start_time = time.time()
country_start_time = time.time()
relations_start_time = time.time()

main_df = pd.read_csv("Resources/netflix_titles.csv")
main_df = main_df.dropna()
main_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...
5,s6,TV Show,46,Serdar Akar,"Erdal Beşikçioğlu, Yasemin Allen, Melis Birkan...",Turkey,"July 1, 2017",2016,TV-MA,1 Season,"International TV Shows, TV Dramas, TV Mysteries",A genetics professor experiments with a treatm...


In [3]:
cleaned_df = main_df.rename(columns={'listed_in':'genre', 'cast':'actor'})

cleaned_df.actor = cleaned_df.actor.str.split(', ')
cleaned_df = cleaned_df.explode('actor').reset_index(drop=True)

cleaned_df.country = cleaned_df.country.str.split(', ')
cleaned_df = cleaned_df.explode('country').reset_index(drop=True)

cleaned_df.genre = cleaned_df.genre.str.split(', ')
cleaned_df = cleaned_df.explode('genre').reset_index(drop=True)

cleaned_df

,show_id,type,title,director,actor,country,date_added,release_year,rating,duration,genre,description
0,s2,Movie,7:19,Jorge Michel Grau,Demián Bichir,Mexico,"December 23, 2016",2016,TV-MA,93 min,Dramas,After a devastating earthquake hits Mexico Cit...
1,s2,Movie,7:19,Jorge Michel Grau,Demián Bichir,Mexico,"December 23, 2016",2016,TV-MA,93 min,International Movies,After a devastating earthquake hits Mexico Cit...
2,s2,Movie,7:19,Jorge Michel Grau,Héctor Bonilla,Mexico,"December 23, 2016",2016,TV-MA,93 min,Dramas,After a devastating earthquake hits Mexico Cit...
3,s2,Movie,7:19,Jorge Michel Grau,Héctor Bonilla,Mexico,"December 23, 2016",2016,TV-MA,93 min,International Movies,After a devastating earthquake hits Mexico Cit...
4,s2,Movie,7:19,Jorge Michel Grau,Oscar Serrano,Mexico,"December 23, 2016",2016,TV-MA,93 min,Dramas,After a devastating earthquake hits Mexico Cit...
...,...,...,...,...,...,...,...,...,...,...,...,...
113131,s7784,Movie,Zubaan,Mozez Singh,Anita Shabdish,India,"March 2, 2019",2015,TV-14,111 min,International Movies,A scrappy but poor boy worms his way into a ty...
113132,s7784,Movie,Zubaan,Mozez Singh,Anita Shabdish,India,"March 2, 2019",2015,TV-14,111 min,Music & Musicals,A scrappy but poor boy worms his way into a ty...
113133,s7784,Movie,Zubaan,Mozez Singh,Chittaranjan Tripathy,India,"March 2, 2019",2015,TV-14,111 min,Dramas,A scrappy but poor boy worms his way into a ty...
113134,s7784,Movie,Zubaan,Mozez Singh,Chittaranjan Tripathy,India,"March 2, 2019",2015,TV-14,111 min,International Movies,A scrappy but poor boy worms his way into a ty...


In [4]:
# Updated ERD
# shows
# -
# show_id VARCHAR(255) PK
# title VARCHAR(255)
# rating VARCHAR(255)
# description VARCHAR(255)
# director VARCHAR(255)
# type VARCHAR(255)
# date_added VARCHAR(255)
# year INT


# actors
# -
# actor_id INT PK
# actor VARCHAR(255)


# countries
# -
# country_id INT PK 
# country VARCHAR(255)


# genres
# -
# genre_id INT PK
# genre VARCHAR(255) 


# relations
# -
# show_id VARCHAR(255) FK >- shows.show_id
# actor_id INT FK >-< actors.actor_id
# country_id INT FK >-< countries.country_id
# genre_id INT FK >-< genres.genre_id

In [5]:
# Create table for shows
try:
    show_df = cleaned_df[["show_id", "title", "director", "rating", "type", "date_added", "release_year", "description", ]]
    show_df = show_df.drop_duplicates()

    show_df["date_added"] = pd.to_datetime(show_df["date_added"])

    show_df.to_csv('Resources/shows.csv', index=False)

except:
    show_errors+=1


# show_df["date_added"]
# remove year
show_count=show_df.shape[0]
show_df

,show_id,title,director,rating,type,date_added,release_year,description
0,s2,7:19,Jorge Michel Grau,TV-MA,Movie,2016-12-23,2016,After a devastating earthquake hits Mexico Cit...
12,s3,23:59,Gilbert Chan,R,Movie,2018-12-20,2011,"When an army recruit is found dead, his fellow..."
30,s4,9,Shane Acker,PG-13,Movie,2017-11-16,2009,"In a postapocalyptic world, rag-doll robots hi..."
57,s5,21,Robert Luketic,PG-13,Movie,2020-01-01,2008,A brilliant group of students become card-coun...
69,s6,46,Serdar Akar,TV-MA,TV Show,2017-07-01,2016,A genetics professor experiments with a treatm...
...,...,...,...,...,...,...,...,...
112992,s7779,Zombieland,Ruben Fleischer,R,Movie,2019-11-01,2009,Looking to survive in a world taken over by zo...
113006,s7781,Zoo,Shlok Sharma,TV-MA,Movie,2018-07-01,2018,A drug dealer starts having doubts about his t...
113024,s7782,Zoom,Peter Hewitt,PG,Movie,2020-01-11,2006,"Dragged from civilian life, a former superhero..."
113042,s7783,Zozo,Josef Fares,TV-MA,Movie,2020-10-19,2005,When Lebanon's Civil War deprives Zozo of his ...


In [6]:
# Create table for actors and transform
try:
    actor_df = cleaned_df[["actor"]]

    actor_df = actor_df.drop_duplicates()

    actor_df.to_csv('Resources/actors.csv')   
except:
    actors_errors+=1
actor_count=actor_df.shape[0]
actor_df

,actor
0,Demián Bichir
2,Héctor Bonilla
4,Oscar Serrano
6,Azalia Ortiz
8,Octavio Michel
...,...
113118,Raaghav Chanana
113124,Meghna Malik
113127,Malkeet Rauni
113130,Anita Shabdish


In [7]:
# Create table for countries and transform
try:
    country_df = cleaned_df[["country"]]

    country_df = country_df.drop_duplicates()

    country_df.to_csv('Resources/countries.csv')
    
except:
    country_errors+=1
country_count=country_df.shape[0]
country_df

,country
0,Mexico
12,Singapore
30,United States
69,Turkey
99,Egypt
...,...
97863,Somalia
97865,Sudan
98627,Panama
110854,East Germany


In [8]:
# Create table for genres and transform
try:
    genre_df = cleaned_df[["genre"]]

    genre_df = genre_df.drop_duplicates()

    genre_df.to_csv('Resources/genres.csv', index=False)
    
except:
    genre_errors+=1
genre_count=genre_df.shape[0]
genre_df

,genre
0,Dramas
1,International Movies
12,Horror Movies
30,Action & Adventure
31,Independent Movies
32,Sci-Fi & Fantasy
69,International TV Shows
70,TV Dramas
71,TV Mysteries
140,Thrillers


In [9]:
connection_string = "postgres:postgres@localhost:5432/netflix3_db"
engine = create_engine(f'postgresql://{connection_string}')

In [10]:
conn = engine.connect()

In [11]:
engine.table_names()

['shows', 'relations', 'actors', 'countries', 'genres']

In [12]:
show_df.to_sql(name='shows', con=engine, if_exists='append', index=False)

In [13]:
actor_df.to_sql(name='actors', con=engine, if_exists='append', index=False)

In [14]:
country_df.to_sql(name='countries', con=engine, if_exists='append', index=False)

In [15]:
genre_df.to_sql(name='genres', con=engine, if_exists='append', index=False)

In [16]:
genre_sql = pd.read_sql('SELECT * FROM genres', conn)
genre_sql
genre_end_time=time.time() - genre_start_time

In [17]:
country_sql = pd.read_sql('SELECT * FROM countries', conn)
country_sql
country_end_time=time.time() - country_start_time

In [18]:
actor_sql = pd.read_sql('SELECT * FROM actors', conn)
actor_sql
actor_end_time=time.time() - actor_start_time

In [19]:
show_sql = pd.read_sql('SELECT * FROM shows', conn)
show_sql
shows_end_time=time.time() - show_start_time

In [20]:
try:
    relations_df = pd.merge(cleaned_df, actor_sql, on='actor', how = 'outer')

    relations_df = relations_df.rename(columns={'id':'actor_id'})
    relations_df
except:
    relations_errors+=1

In [21]:
try:
    relations_df = pd.merge(relations_df, genre_sql, on='genre', how = 'outer')

    relations_df = relations_df.rename(columns={'id':'genre_id'})
    relations_df
except:
    relations_errors+=1


In [22]:
try:
    relations_df = pd.merge(relations_df, country_sql, on='country', how = 'outer')

    relations_df = relations_df.rename(columns={'id':'country_id'}) 
except:
    relations_errors+=1


In [23]:
try:
    relations_df = relations_df[['show_id', 'actor_id', 'genre_id', 'country_id']]
except:
    relations_errors+=1
relations_count=relations_df.shape[0]
relations_df

,show_id,actor_id,genre_id,country_id
0,s2,1,1,1
1,s5527,1,1,1
2,s2,2,1,1
3,s2,3,1,1
4,s2,4,1,1
...,...,...,...,...
113131,s4148,14291,11,89
113132,s4148,14292,11,89
113133,s4148,14293,11,89
113134,s4148,14294,11,89


In [24]:
relations_df.to_sql(name='relations', con=engine, if_exists='append', index=False)
relations_end_time=time.time() - relations_start_time

In [25]:
relations_df.to_csv('Resources/relations.csv', index=False)

In [26]:
etl_report=pd.DataFrame({'domain' : ['genre', 'actor','show','country','relation'],
                       'row_counts' : [genre_count, actor_count,show_count,country_count,relations_count],
                        'processing_time':[str(round(genre_end_time,2)) + ' seconds', str(round(actor_end_time,2)) + ' seconds',str(round(shows_end_time,2)) + ' seconds',str(round(country_end_time,2)) + ' seconds',str(round(relations_end_time,2)) + ' seconds'],
                         'error_count':[genre_errors, actor_errors,show_errors,country_errors,relations_errors]})

In [27]:
etl_report

,domain,row_counts,processing_time,error_count
0,genre,42,5.43 seconds,0
1,actor,23215,5.48 seconds,0
2,show,4808,5.52 seconds,0
3,country,105,5.44 seconds,0
4,relation,113136,26.92 seconds,0
